# Graph Analytics: Project

/

#### 2021/2022
#### Lorenzo Pigozzi

In [1]:
# importing the libraries
import pandas as pd
import numpy as np 
import networkx as nx
import itertools
import collections
import random
import matplotlib.pyplot as plt
# network related libraries
import pm4py
import pycelonis
import networkx as nx

import warnings
warnings.filterwarnings("ignore")

## 1. Importing the data

In [2]:
# importing the datasets
brazil = pd.read_csv('ACTIVITY_TABLE_BR.csv')
# germany = pd.read_csv('ACTIVITY_TABLE_DE.csv')
# portugal = pd.read_csv('ACTIVITY_TABLE_PT.csv')
# thailand = pd.read_csv('ACTIVITY_TABLE_TH.csv')

In [134]:
# to remove in future
df = brazil.copy()

In [109]:
df.head()

ACTIVITY            TIMESTAMP  SALES_DOC  \
0                 Create Sales Order  2021-10-04 16:22:36   50043243   
1            Create Sales Order Item  2021-10-04 16:22:36   50043243   
2  Material Availability Date passed  2021-10-04 00:00:00   50043243   
3                 Create Sales Order  2021-10-04 16:22:36   50043243   
4            Create Sales Order Item  2021-10-04 16:22:36   50043243   

   SALES_DOC_ITEM COUNTRY  
0              10      BR  
1              10      BR  
2              10      BR  
3              20      BR  
4              20      BR

In [110]:
df.shape

(1837252, 5)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1837252 entries, 0 to 1837251
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   ACTIVITY        object
 1   TIMESTAMP       object
 2   SALES_DOC       int64 
 3   SALES_DOC_ITEM  int64 
 4   COUNTRY         object
dtypes: int64(2), object(3)
memory usage: 70.1+ MB


## 2. Data exploration

In [135]:
# changing data type for SO number and Item number
df['SALES_DOC'] = df['SALES_DOC'].astype(str)
df['SALES_DOC_ITEM'] = df['SALES_DOC_ITEM'].astype(str)

In [136]:
# creating the CASE KEY, combination of Sales Order and item
df['CASE_KEY'] = df['SALES_DOC'] + df['SALES_DOC_ITEM']

In [84]:
df.head()

ACTIVITY            TIMESTAMP SALES_DOC  \
0                 Create Sales Order  2021-10-04 16:22:36  50043243   
1            Create Sales Order Item  2021-10-04 16:22:36  50043243   
2  Material Availability Date passed  2021-10-04 00:00:00  50043243   
3                 Create Sales Order  2021-10-04 16:22:36  50043243   
4            Create Sales Order Item  2021-10-04 16:22:36  50043243   

  SALES_DOC_ITEM COUNTRY    CASE_KEY  
0             10      BR  5004324310  
1             10      BR  5004324310  
2             10      BR  5004324310  
3             20      BR  5004324320  
4             20      BR  5004324320

In [11]:
# number of distinct SO in the log
print('Number of distinct SO in the log: ', len(df['SALES_DOC'].unique()))

Number of distinct SO in the log:  90205


In [12]:
# number of distinct cases in the log
print('Number of distinct cases in the log: ', len(df['CASE_KEY'].unique()))

Number of distinct cases in the log:  365032


In [13]:
df.columns

Index(['ACTIVITY', 'TIMESTAMP', 'SALES_DOC', 'SALES_DOC_ITEM', 'COUNTRY',
       'CASE_KEY'],
      dtype='object')

In [14]:
df['ACTIVITY'].unique()

array(['Create Sales Order', 'Create Sales Order Item',
       'Material Availability Date passed', 'Record Goods Issue',
       'Create Delivery', 'Create Invoice', 'Create Picking',
       'Change Material', 'Change Price', 'Change Payment Terms',
       'Create Purchase Order', 'Create Pro forma invoice',
       'Cancel Goods Issue', 'Record Return Goods Receipt',
       'Create Returns delivery for order', 'Create Credit memo',
       'Clear Invoice', 'Change Requested Quantity',
       'Set Reason for Rejection', 'Change Inco Terms (Part 2)',
       'Create Invoice cancellation', 'Change Inco Terms (Part 1)',
       'Change Requested Goods Issue Date',
       'Change Material Availability Date', 'Customer Acceptance (L650)',
       'Change Requested Delivery Date', 'Change Confirmed Delivery Date',
       'Change Confirmed Goods Issue Date', 'Change Confirmed Quantity',
       'Set Initial Delivery Block', 'Change Item Category',
       'Remove Delivery Block', 'Change Shipping Po

## 3. Data Engineering

### 3.1. Weight calculation

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1837252 entries, 0 to 1837251
Data columns (total 6 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   ACTIVITY        object
 1   TIMESTAMP       object
 2   SALES_DOC       object
 3   SALES_DOC_ITEM  object
 4   COUNTRY         object
 5   CASE_KEY        object
dtypes: object(6)
memory usage: 84.1+ MB


In [139]:
## to obtain the weights of the graph, it is necessary to engineer and make calculation for the time

# changing the date type
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], format='%Y-%m-%d %H:%M:%S')

# calculating the time difference after sorting by timestamp and grouping by CASE_KEY
df['diff'] = df.sort_values('TIMESTAMP', ascending = True).groupby('CASE_KEY')['TIMESTAMP'].diff()


In [114]:
# weight_hour: getting the difference in seconds and dividing for 3600 to get the number of hours
# 3600 seconds = 1 hour
df['weight_hour'] = df['diff'].dt.total_seconds() / 3600

In [115]:
# dropping unnecessary columns
df.drop(columns=['SALES_DOC', 'SALES_DOC_ITEM', 'diff'], inplace = True)

In [128]:
df = df.sort_values(['CASE_KEY', 'TIMESTAMP', 'ACTIVITY'], ascending = True).reset_index(drop = True)

In [130]:
backup = df.copy()
# df = backup.copy()

In [129]:
df.head(50)

ACTIVITY           TIMESTAMP COUNTRY  \
0                  Create Sales Order 2020-07-24 00:30:07      BR   
1             Create Sales Order Item 2020-07-24 00:30:07      BR   
2   Material Availability Date passed 2020-10-21 00:00:00      BR   
3                  Create Sales Order 2020-07-24 00:30:07      BR   
4             Create Sales Order Item 2020-07-24 00:30:07      BR   
5   Material Availability Date passed 2020-10-21 00:00:00      BR   
6                  Create Sales Order 2020-07-24 00:30:07      BR   
7             Create Sales Order Item 2020-07-24 00:30:07      BR   
8   Material Availability Date passed 2020-10-21 00:00:00      BR   
9                  Create Sales Order 2020-07-24 00:30:07      BR   
10            Create Sales Order Item 2020-07-24 00:30:24      BR   
11  Material Availability Date passed 2020-10-21 00:00:00      BR   
12                 Create Sales Order 2020-07-24 00:36:13      BR   
13            Create Sales Order Item 2020-07-24 00:36:13      BR   
14  Material Availability Date passed 2020-10-21 00:00:00      BR   
15                 Create Sales Order 2020-07-24 00:36:13      BR   
16            Create Sales Order Item 2020-07-24 00:36:13      BR   
17  Material Availability Date passed 2020-10-21 00:00:00      BR   
18                 Create Sales Order 2020-07-24 00:36:13      BR   
19            Create Sales Order Item 2020-07-24 00:36:34      BR   
20  Material Availability Date passed 2020-10-21 00:00:00      BR   
21                 Create Sales Order 2020-07-24 00:51:53      BR   
22            Create Sales Order Item 2020-07-24 00:51:54      BR   
23  Material Availability Date passed 2020-10-21 00:00:00      BR   
24                 Create Sales Order 2020-07-24 00:51:53      BR   
25            Create Sales Order Item 2020-07-24 00:51:54      BR   
26  Material Availability Date passed 2020-10-21 00:00:00      BR   
27                 Create Sales Order 2020-07-24 00:51:53      BR   
28            Create Sales Order Item 2020-07-24 00:52:13      BR   
29  Material Availability Date passed 2020-10-21 00:00:00      BR   
30                 Create Sales Order 2020-07-24 14:14:19      BR   
31            Create Sales Order Item 2020-07-24 14:14:19      BR   
32                     Create Picking 2020-10-16 00:00:00      BR   
33  Material Availability Date passed 2020-10-16 00:00:00      BR   
34                 Create Sales Order 2020-07-24 14:14:19      BR   
35            Create Sales Order Item 2020-07-24 14:14:20      BR   
36  Material Availability Date passed 2020-10-16 00:00:00      BR   
37                 Create Sales Order 2020-07-24 14:14:19      BR   
38                     Create Picking 2020-10-16 00:00:00      BR   
39  Material Availability Date passed 2020-10-16 00:00:00      BR   
40            Create Sales Order Item 2020-10-16 14:37:58      BR   
41                 Create Sales Order 2020-07-24 14:14:51      BR   
42            Create Sales Order Item 2020-07-24 14:14:51      BR   
43  Material Availability Date passed 2020-07-27 00:00:00      BR   
44                 Create Sales Order 2020-07-24 14:14:51      BR   
45            Create Sales Order Item 2020-07-24 14:14:51      BR   
46  Material Availability Date passed 2020-07-27 00:00:00      BR   
47                 Create Sales Order 2020-07-24 14:14:51      BR   
48            Create Sales Order Item 2020-07-24 14:14:51      BR   
49  Material Availability Date passed 2020-07-27 00:00:00      BR   

        CASE_KEY  weight_hour  
0   112632797410          NaN  
1   112632797410     0.000000  
2   112632797410  2135.498056  
3   112632797420          NaN  
4   112632797420     0.000000  
5   112632797420  2135.498056  
6   112632797430     0.000000  
7   112632797430          NaN  
8   112632797430  2135.498056  
9   112632797440          NaN  
10  112632797440     0.004722  
11  112632797440  2135.493333  
12  112632798010          NaN  
13  112632798010     0.000000  
14  112632798010  2135.396389

### Test

In [67]:
df_1 = df[df['CASE_KEY'] == '112633171510'].sort_values('TIMESTAMP', ascending = True).reset_index(drop=True)

In [69]:
df_1 = df_1[['CASE_KEY', 'ACTIVITY', 'weight_hour']]

In [70]:
source = ['']
for i in range(1, max(df_1.index)+1):
    source.append(df_1['ACTIVITY'][i-1])

df_1['Source'] = source

In [71]:
df_1

CASE_KEY                           ACTIVITY  weight_hour  \
0  112633171510            Create Sales Order Item          NaN   
1  112633171510                 Create Sales Order     0.000000   
2  112633171510  Material Availability Date passed     0.000000   
3  112633171510                     Create Picking  2001.761389   

                              Source  
0                                     
1            Create Sales Order Item  
2                 Create Sales Order  
3  Material Availability Date passed

In [73]:
df_1 = df_1[['CASE_KEY', 'Source', 'ACTIVITY', 'weight_hour']]

In [74]:
df_1.columns = ['CASE_KEY', 'Source', 'Target', 'weight']

In [75]:
df_1

CASE_KEY                             Source  \
0  112633171510                                      
1  112633171510            Create Sales Order Item   
2  112633171510                 Create Sales Order   
3  112633171510  Material Availability Date passed   

                              Target       weight  
0            Create Sales Order Item          NaN  
1                 Create Sales Order     0.000000  
2  Material Availability Date passed     0.000000  
3                     Create Picking  2001.761389

In [77]:
len(df['CASE_KEY'].unique())

365032

### Test 2

In [131]:
df.head()

ACTIVITY           TIMESTAMP COUNTRY  \
0                 Create Sales Order 2020-07-24 00:30:07      BR   
1            Create Sales Order Item 2020-07-24 00:30:07      BR   
2  Material Availability Date passed 2020-10-21 00:00:00      BR   
3                 Create Sales Order 2020-07-24 00:30:07      BR   
4            Create Sales Order Item 2020-07-24 00:30:07      BR   

       CASE_KEY  weight_hour  
0  112632797410          NaN  
1  112632797410     0.000000  
2  112632797410  2135.498056  
3  112632797420          NaN  
4  112632797420     0.000000

In [132]:
df = df[['CASE_KEY', 'ACTIVITY', 'weight_hour']]

In [133]:
df.head(50)

CASE_KEY                           ACTIVITY  weight_hour
0   112632797410                 Create Sales Order          NaN
1   112632797410            Create Sales Order Item     0.000000
2   112632797410  Material Availability Date passed  2135.498056
3   112632797420                 Create Sales Order          NaN
4   112632797420            Create Sales Order Item     0.000000
5   112632797420  Material Availability Date passed  2135.498056
6   112632797430                 Create Sales Order     0.000000
7   112632797430            Create Sales Order Item          NaN
8   112632797430  Material Availability Date passed  2135.498056
9   112632797440                 Create Sales Order          NaN
10  112632797440            Create Sales Order Item     0.004722
11  112632797440  Material Availability Date passed  2135.493333
12  112632798010                 Create Sales Order          NaN
13  112632798010            Create Sales Order Item     0.000000
14  112632798010  Material Availability Date passed  2135.396389
15  112632798020                 Create Sales Order          NaN
16  112632798020            Create Sales Order Item     0.000000
17  112632798020  Material Availability Date passed  2135.396389
18  112632798030                 Create Sales Order          NaN
19  112632798030            Create Sales Order Item     0.005833
20  112632798030  Material Availability Date passed  2135.390556
21  112632798710                 Create Sales Order          NaN
22  112632798710            Create Sales Order Item     0.000278
23  112632798710  Material Availability Date passed  2135.135000
24  112632798720                 Create Sales Order          NaN
25  112632798720            Create Sales Order Item     0.000278
26  112632798720  Material Availability Date passed  2135.135000
27  112632798730                 Create Sales Order          NaN
28  112632798730            Create Sales Order Item     0.005556
29  112632798730  Material Availability Date passed  2135.129722
30  112633171510                 Create Sales Order     0.000000
31  112633171510            Create Sales Order Item          NaN
32  112633171510                     Create Picking  2001.761389
33  112633171510  Material Availability Date passed     0.000000
34  112633171520                 Create Sales Order          NaN
35  112633171520            Create Sales Order Item     0.000278
36  112633171520  Material Availability Date passed  2001.761111
37  112633171530                 Create Sales Order          NaN
38  112633171530                     Create Picking  2001.761389
39  112633171530  Material Availability Date passed     0.000000
40  112633171530            Create Sales Order Item    14.632778
41  112633172010                 Create Sales Order          NaN
42  112633172010            Create Sales Order Item     0.000000
43  112633172010  Material Availability Date passed    57.752500
44  112633172020                 Create Sales Order     0.000000
45  112633172020            Create Sales Order Item          NaN
46  112633172020  Material Availability Date passed    57.752500
47  112633172030                 Create Sales Order     0.000000
48  112633172030            Create Sales Order Item          NaN
49  112633172030  Material Availability Date passed    57.752500

In [95]:
source = ['']
for i in df['CASE_KEY'].unique():
    df_1 = df[df['CASE_KEY'] == i].reset_index(drop=True)
    for j in range(1, max(df_1.index)+1):
        source.append(df_1['ACTIVITY'][j-1])

df['Source'] = source

KeyboardInterrupt: 

In [ ]:
df

In [ ]:
df.groupby('CASE_KEY').sort_values('TIMESTAMP', ascending = True)

In [96]:
df

CASE_KEY                           ACTIVITY  weight_hour
1044006  112632797410                 Create Sales Order          NaN
1044007  112632797410            Create Sales Order Item     0.000000
1044008  112632797410  Material Availability Date passed  2135.498056
1044009  112632797420                 Create Sales Order          NaN
1044016  112632797420            Create Sales Order Item     0.000000
...               ...                                ...          ...
194756     5004377520                 Create Sales Order     0.000000
194760     5004377610                 Create Sales Order          NaN
194761     5004377610            Create Sales Order Item     0.000000
194759     5004377620                 Create Sales Order     0.000000
194762     5004377620            Create Sales Order Item          NaN

[1837252 rows x 3 columns]

In [97]:
mask = (df.groupby(df['CASE_KEY']).transform(lambda x: x.shift(fill_value=False).cummax()))

In [101]:
backup = df.copy()

In [102]:
source = ['']
for i in range(1, max(df.index)+1):
    source.append(df['ACTIVITY'][i-1])

df['Source'] = source

In [107]:
df.head(50)

CASE_KEY                           ACTIVITY  weight_hour  \
1044006  112632797410                 Create Sales Order          NaN   
1044007  112632797410            Create Sales Order Item     0.000000   
1044008  112632797410  Material Availability Date passed  2135.498056   
1044009  112632797420                 Create Sales Order          NaN   
1044016  112632797420            Create Sales Order Item     0.000000   
1044015  112632797420  Material Availability Date passed  2135.498056   
1044010  112632797430                 Create Sales Order     0.000000   
1044011  112632797430            Create Sales Order Item          NaN   
1044012  112632797430  Material Availability Date passed  2135.498056   
1044013  112632797440                 Create Sales Order          NaN   
1044014  112632797440            Create Sales Order Item     0.004722   
1044005  112632797440  Material Availability Date passed  2135.493333   
1044018  112632798010            Create Sales Order Item     0.000000   
1044019  112632798010                 Create Sales Order          NaN   
1044017  112632798010  Material Availability Date passed  2135.396389   
1044024  112632798020            Create Sales Order Item     0.000000   
1044025  112632798020                 Create Sales Order          NaN   
1044023  112632798020  Material Availability Date passed  2135.396389   
1044020  112632798030                 Create Sales Order          NaN   
1044021  112632798030            Create Sales Order Item     0.005833   
1044022  112632798030  Material Availability Date passed  2135.390556   
1044031  112632798710                 Create Sales Order          NaN   
1044030  112632798710            Create Sales Order Item     0.000278   
1044029  112632798710  Material Availability Date passed  2135.135000   
1044033  112632798720                 Create Sales Order          NaN   
1044028  112632798720            Create Sales Order Item     0.000278   
1044027  112632798720  Material Availability Date passed  2135.135000   
1044034  112632798730                 Create Sales Order          NaN   
1044026  112632798730            Create Sales Order Item     0.005556   
1044032  112632798730  Material Availability Date passed  2135.129722   
180322   112633171510            Create Sales Order Item          NaN   
180324   112633171510                 Create Sales Order     0.000000   
180321   112633171510  Material Availability Date passed     0.000000   
180323   112633171510                     Create Picking  2001.761389   
180326   112633171520                 Create Sales Order          NaN   
180325   112633171520            Create Sales Order Item     0.000278   
180327   112633171520  Material Availability Date passed  2001.761111   
180329   112633171530                 Create Sales Order          NaN   
180320   112633171530  Material Availability Date passed     0.000000   
180328   112633171530                     Create Picking  2001.761389   
180319   112633171530            Create Sales Order Item    14.632778   
180335   112633172010            Create Sales Order Item     0.000000   
180338   112633172010                 Create Sales Order          NaN   
180334   112633172010  Material Availability Date passed    57.752500   
180332   112633172020            Create Sales Order Item          NaN   
180333   112633172020                 Create Sales Order     0.000000   
180331   112633172020  Material Availability Date passed    57.752500   
180336   112633172030            Create Sales Order Item          NaN   
180337   112633172030                 Create Sales Order     0.000000   
180330   112633172030  Material Availability Date passed    57.752500   

                                    Source  
1044006                                     
1044007                 Create Sales Order  
1044008            Create Sales Order Item  
1044009  Material Availability Date passed  
1044016                 Create Sales Order  
1044015            

In [106]:
df[~df.groupby('CASE_KEY')['weight_hour'].cumsum().eq(0)]

CASE_KEY                           ACTIVITY  weight_hour  \
1044006  112632797410                 Create Sales Order          NaN   
1044008  112632797410  Material Availability Date passed  2135.498056   
1044009  112632797420                 Create Sales Order          NaN   
1044015  112632797420  Material Availability Date passed  2135.498056   
1044011  112632797430            Create Sales Order Item          NaN   
...               ...                                ...          ...   
194747     5004377460            Create Sales Order Item     0.000278   
194758     5004377510                 Create Sales Order          NaN   
194755     5004377520            Create Sales Order Item          NaN   
194760     5004377610                 Create Sales Order          NaN   
194762     5004377620            Create Sales Order Item          NaN   

                                    Source  
1044006                                     
1044008            Create Sales Order Item  
1044009  Material Availability Date passed  
1044015            Create Sales Order Item  
1044011                 Create Sales Order  
...                                    ...  
194747                  Create Sales Order  
194758             Create Sales Order Item  
194755                  Create Sales Order  
194760                  Create Sales Order  
194762             Create Sales Order Item  

[1708259 rows x 4 columns]

In [105]:
df[df.index>=df.groupby('CASE_KEY').ACTIVITY.transform('idxmax')]

TypeError: reduction operation 'argmax' not allowed for this dtype

Note: explore https://pm4py.fit.fraunhofer.de/

In [41]:
# ## From the lab5 notebook

# documents_dict= dict()
# for document in df.SALES_DOC_ITEM.unique():
#     documents_dict[document]=len(df[df.SALES_DOC_ITEM == document].ACTIVITY.unique())

# documents_dict

In [42]:
# #number of different activities per document
# import matplotlib.pyplot as plt

# plt.scatter(*zip(*documents_dict.items()));